# Feature creation and translation to wide df

In [ ]:
######################## LOAD IN FILES #############################
import os
import pandas as pd
import sys
import importlib

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Define data directory
brighten_dir = os.path.join(project_root, 'BRIGHTEN_data')

# Add project root to sys.path for script usage
sys.path.append(project_root)

# Import and reload (optional) custom scripts
from scripts import preprocessing as pre
from scripts import visualization as vis
importlib.reload(pre)
importlib.reload(vis)

df_alldays_int70 = pd.read_csv(os.path.join(brighten_dir, 'df_alldays_int70.csv'))
week_df_int70 = pd.read_csv(os.path.join(brighten_dir, 'week_df_int70.csv'))


## Weekly cols
phq9_cols = ['phq9_1','phq9_2','phq9_3','phq9_4','phq9_5','phq9_6','phq9_7','phq9_8','phq9_9','phq9_sum','phq9_cat']
sds_cols = ['sds_1','sds_2','sds_3','stress','support']
sleep_cols = ['sleep_1','sleep_2','sleep_3']
gic_cols = ['mood_1']

phq2_cols = ['phq2_1','phq2_2','phq2_sum']

daily_cols_v1 = ['aggregate_communication', 'call_count',
       'call_duration', 'interaction_diversity', 'missed_interactions',
       'mobility', 'mobility_radius', 'sms_count', 'sms_length',
       'unreturned_calls']


daily_cols_v2 = ['distance_walking', 'hours_active',
       'hours_high_speed_transportation', 'hours_of_sleep',
       'hours_powered_vehicle', 'hours_stationary', 'hours_stationary_nhw',
       'hours_walking', 'location_variance',
        'cloud_cover_mean','dew_point_mean',
        'humidity_mean','temp_mean','dew_point_IQR','humidity_IQR',
        'temp_IQR','cloud_cover_IQR','cloud_cover_std','dew_point_std',
        'humidity_std','temp_std','cloud_cover_median','dew_point_median',
        'humidity_median','temp_median']


daily_cols_v3 = ['callDuration_incoming',
       'callDuration_outgoing', 'textCount', 'textCount_received',
       'textCount_sent', 'textLength_received', 'textLength_sent',
       'uniqueNumbers_calls_incoming', 'uniqueNumbers_calls_missed',
       'uniqueNumbers_calls_outgoing', 'uniqueNumbers_texts',
       'uniqueNumbers_texts_received', 'uniqueNumbers_texts_sent']

passive_cols = daily_cols_v1 + daily_cols_v2 + daily_cols_v3

added_columns = ['phq9_sum_change', 'phq9_cat_change']

mobility_cols = ['mobility', 'mobility_radius']




/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/3972348359.py:6: DtypeWarning: Columns (14,22,47,105,106,107,117,118,119,120,121,122,123,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df_alldays_int70 = pd.read_csv(os.path.join(brighten_dir, 'df_alldays_int70.csv'))
/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/3972348359.py:7: DtypeWarning: Columns (23,106,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  week_df_int70 = pd.read_csv(os.path.join(brighten_dir, 'week_df_int70.csv'))


In [ ]:
id_columns = ['dt','week','num_id','participant_id']

# dt
# Turn dt to datetime
df_names = ['v1_day', 'v2_day', 'v1_week_phq9', 'v2_week_phq9']
id_columns = ['dt','week','num_id','participant_id']

for name in df_names:
    df_pca = pd.read_csv(os.path.join(brighten_dir, f'{name}_pca.csv'))
    if 'day' in name:
        df_bin = df_pca.merge(df_alldays_int70[id_columns + ['phq2_bin']], on=id_columns, how='left')
    if 'week' in name:
        df_bin = df_pca.merge(df_alldays_int70[id_columns + ['phq2_bin', 'phq9_bin','phq9_cat']], on=id_columns, how='left')

    df_bin.to_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))





In [ ]:
### making features -- averaging of time data
id_columns = ['dt','week','num_id','participant_id','day']



for name in df_names:
    df_pca_bin = pd.read_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))
    wide_df_pca_bin = pre.make_wide_df(df_pca_bin, id_columns, brighten_dir, f'wide_{name}_bin')






FOR FEATURE:  sms_count


/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/1590456300.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope_of_feature(g, x_col="day", y_col=feature))
/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/1590456300.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope_of_feature(g, x_col="day", y_col=feature))



FOR FEATURE:  phq2_sum

FOR FEATURE:  unreturned_calls


/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/1590456300.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope_of_feature(g, x_col="day", y_col=feature))
/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/1590456300.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope_of_feature(g, x_col="day", y_col=feature))
/var


FOR FEATURE:  interaction_diversity


...

/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/1590456300.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope_of_feature(g, x_col="day", y_col=feature))
/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/1590456300.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope_of_feature(g, x_col="day", y_col=feature))
/var


FOR FEATURE:  distance_walking

FOR FEATURE:  hours_powered_vehicle

FOR FEATURE:  phq2_bin


/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/1590456300.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope_of_feature(g, x_col="day", y_col=feature))
/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/1590456300.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope_of_feature(g, x_col="day", y_col=feature))
/var


FOR FEATURE:  pc_vehicle

FOR FEATURE:  phq2_2

FOR FEATURE:  pc_depression

FOR FEATURE:  hours_walking

FOR FEATURE:  phq2_1


/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/1590456300.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope_of_feature(g, x_col="day", y_col=feature))
/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/1590456300.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope_of_feature(g, x_col="day", y_col=feature))


In [ ]:

# Add binary
for name in df_names:
    df_pca_bin = pd.read_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))
    if 'day' in name:
        df_pca_bin_day = pre.round_vars_phq2(df_pca_bin)
        df_pca_bin_day = df_pca_bin_day.dropna()
        df_pca_bin_day.to_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))
    if 'week' in name:
        df_pca_bin_week = pre.round_vars_phq9(df_pca_bin)
        df_pca_bin_week.to_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))


In [ ]:
id_columns = ['dt','week','num_id','participant_id','day']
demo_clusters = pd.read_csv(os.path.join(brighten_dir, 'demo_clusters.csv'))

# Merge in demographic clusters
for name in df_names:
    df_bin = pd.read_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))
    df_clusters = df_bin.merge(demo_clusters[['participant_id', 'Cluster']], on=['participant_id'], how='left')
    df_clusters = df_clusters.merge(df_alldays_int70[['participant_id', 'phq9_base_cluster']], on=['participant_id'], how='left')
    df_clusters = df_clusters.drop_duplicates().dropna()
    df_clusters.to_csv(os.path.join(brighten_dir, f'wide_{name}_clusters.csv'))
